# Transaction Management

Generally, SQLiteDB tries to manage transactions to be as efficient as possible. When methods allow for multiple actions (e.g. `insertmany(items)` or `insert(*items)`) it uses `sqlite3`'s optimized interface. But other actions can be improved using the context manager.

## Aside: Logging SQL Statements

SQL statements can be debug-logged with its own Python logger. This is turned off by default as it is beyond even DEBUG level information but can be turned on with the `JSONLiteDB_SQL_DEBUG` environment variable.

In [1]:
import logging

In [2]:
import os, sys

p = os.path.abspath("../")
if p not in sys.path:
    sys.path.append(p)

import jsonlitedb
from jsonlitedb import JSONLiteDB, Q

In [3]:
os.environ["JSONLiteDB_SQL_DEBUG"] = "true"
logging.basicConfig(level=logging.DEBUG)

In [4]:
db = JSONLiteDB(":memory:")

DEBUG:jsonlitedb:self.table = 'items'
DEBUG:jsonlitedb:DB does not exists
DEBUG:jsonlitedb-sql:
CREATE TABLE IF NOT EXISTS items(
    rowid INTEGER PRIMARY KEY AUTOINCREMENT,
    data JSON
)
DEBUG:jsonlitedb-sql:
CREATE TABLE IF NOT EXISTS items_kv(
    key TEXT PRIMARY KEY,
    val BLOB
)
DEBUG:jsonlitedb-sql:BEGIN 
DEBUG:jsonlitedb-sql:
INSERT OR IGNORE INTO items_kv VALUES ('created','2024-12-27T15:36:26.177603-07:00')

DEBUG:jsonlitedb-sql:
INSERT OR IGNORE INTO items_kv VALUES ('version','0.1.0')

DEBUG:jsonlitedb-sql:COMMIT


## Inserts

Insert items. There will be duplicates but that is fine for this example. Notice in the output when they are commited

In [5]:
items = [
    {"first": "John", "last": "Lennon", "born": 1940, "role": "guitar"},
    {"first": "Paul", "last": "McCartney", "born": 1942, "role": "bass"},
    {"first": "George", "last": "Harrison", "born": 1943, "role": "guitar"},
    {"first": "Ringo", "last": "Starr", "born": 1940, "role": "drums"},
    {"first": "George", "last": "Martin", "born": 1926, "role": "producer"},
]

### Optimized

In [6]:
db.insert(*items)

DEBUG:jsonlitedb-sql:BEGIN 
DEBUG:jsonlitedb-sql:
INSERT  INTO items (data)
VALUES (JSON('{"first": "John", "last": "Lennon", "born": 1940, "role": "guitar"}'))

DEBUG:jsonlitedb-sql:
INSERT  INTO items (data)
VALUES (JSON('{"first": "Paul", "last": "McCartney", "born": 1942, "role": "bass"}'))

DEBUG:jsonlitedb-sql:
INSERT  INTO items (data)
VALUES (JSON('{"first": "George", "last": "Harrison", "born": 1943, "role": "guitar"}'))

DEBUG:jsonlitedb-sql:
INSERT  INTO items (data)
VALUES (JSON('{"first": "Ringo", "last": "Starr", "born": 1940, "role": "drums"}'))

DEBUG:jsonlitedb-sql:
INSERT  INTO items (data)
VALUES (JSON('{"first": "George", "last": "Martin", "born": 1926, "role": "producer"}'))

DEBUG:jsonlitedb-sql:COMMIT


In [7]:
db.insertmany(items)

DEBUG:jsonlitedb-sql:BEGIN 
DEBUG:jsonlitedb-sql:
INSERT  INTO items (data)
VALUES (JSON('{"first": "John", "last": "Lennon", "born": 1940, "role": "guitar"}'))

DEBUG:jsonlitedb-sql:
INSERT  INTO items (data)
VALUES (JSON('{"first": "Paul", "last": "McCartney", "born": 1942, "role": "bass"}'))

DEBUG:jsonlitedb-sql:
INSERT  INTO items (data)
VALUES (JSON('{"first": "George", "last": "Harrison", "born": 1943, "role": "guitar"}'))

DEBUG:jsonlitedb-sql:
INSERT  INTO items (data)
VALUES (JSON('{"first": "Ringo", "last": "Starr", "born": 1940, "role": "drums"}'))

DEBUG:jsonlitedb-sql:
INSERT  INTO items (data)
VALUES (JSON('{"first": "George", "last": "Martin", "born": 1926, "role": "producer"}'))

DEBUG:jsonlitedb-sql:COMMIT


In [8]:
with db:
    for item in items:
        db.insert(item)

DEBUG:jsonlitedb-sql:BEGIN 
DEBUG:jsonlitedb-sql:
INSERT  INTO items (data)
VALUES (JSON('{"first": "John", "last": "Lennon", "born": 1940, "role": "guitar"}'))

DEBUG:jsonlitedb-sql:
INSERT  INTO items (data)
VALUES (JSON('{"first": "Paul", "last": "McCartney", "born": 1942, "role": "bass"}'))

DEBUG:jsonlitedb-sql:
INSERT  INTO items (data)
VALUES (JSON('{"first": "George", "last": "Harrison", "born": 1943, "role": "guitar"}'))

DEBUG:jsonlitedb-sql:
INSERT  INTO items (data)
VALUES (JSON('{"first": "Ringo", "last": "Starr", "born": 1940, "role": "drums"}'))

DEBUG:jsonlitedb-sql:
INSERT  INTO items (data)
VALUES (JSON('{"first": "George", "last": "Martin", "born": 1926, "role": "producer"}'))

DEBUG:jsonlitedb-sql:COMMIT


### NOT optimized

Notice it commits after each

In [9]:
for item in items:
    db.insert(item)

DEBUG:jsonlitedb-sql:BEGIN 
DEBUG:jsonlitedb-sql:
INSERT  INTO items (data)
VALUES (JSON('{"first": "John", "last": "Lennon", "born": 1940, "role": "guitar"}'))

DEBUG:jsonlitedb-sql:COMMIT
DEBUG:jsonlitedb-sql:BEGIN 
DEBUG:jsonlitedb-sql:
INSERT  INTO items (data)
VALUES (JSON('{"first": "Paul", "last": "McCartney", "born": 1942, "role": "bass"}'))

DEBUG:jsonlitedb-sql:COMMIT
DEBUG:jsonlitedb-sql:BEGIN 
DEBUG:jsonlitedb-sql:
INSERT  INTO items (data)
VALUES (JSON('{"first": "George", "last": "Harrison", "born": 1943, "role": "guitar"}'))

DEBUG:jsonlitedb-sql:COMMIT
DEBUG:jsonlitedb-sql:BEGIN 
DEBUG:jsonlitedb-sql:
INSERT  INTO items (data)
VALUES (JSON('{"first": "Ringo", "last": "Starr", "born": 1940, "role": "drums"}'))

DEBUG:jsonlitedb-sql:COMMIT
DEBUG:jsonlitedb-sql:BEGIN 
DEBUG:jsonlitedb-sql:
INSERT  INTO items (data)
VALUES (JSON('{"first": "George", "last": "Martin", "born": 1926, "role": "producer"}'))

DEBUG:jsonlitedb-sql:COMMIT


## Removals

### Optimized

In [10]:
row_ids = [row.rowid for row in db.query(db.Q.last == "Martin")]
db.delete_by_rowid(*row_ids)

DEBUG:jsonlitedb-sql:
SELECT rowid, data FROM items 
WHERE
    ( JSON_EXTRACT(data, '$."last"') = 'Martin' )


DEBUG:jsonlitedb-sql:BEGIN 
DEBUG:jsonlitedb-sql:
DELETE FROM items 
WHERE
    rowid = 5

DEBUG:jsonlitedb-sql:
DELETE FROM items 
WHERE
    rowid = 10

DEBUG:jsonlitedb-sql:
DELETE FROM items 
WHERE
    rowid = 15

DEBUG:jsonlitedb-sql:
DELETE FROM items 
WHERE
    rowid = 20

DEBUG:jsonlitedb-sql:COMMIT


In [11]:
row_ids = [row.rowid for row in db.query(db.Q.last == "Starr")]
with db:
    for row_id in row_ids:
        db.delete_by_rowid(row_id)

DEBUG:jsonlitedb-sql:
SELECT rowid, data FROM items 
WHERE
    ( JSON_EXTRACT(data, '$."last"') = 'Starr' )


DEBUG:jsonlitedb-sql:BEGIN 
DEBUG:jsonlitedb-sql:
DELETE FROM items 
WHERE
    rowid = 4

DEBUG:jsonlitedb-sql:
DELETE FROM items 
WHERE
    rowid = 9

DEBUG:jsonlitedb-sql:
DELETE FROM items 
WHERE
    rowid = 14

DEBUG:jsonlitedb-sql:
DELETE FROM items 
WHERE
    rowid = 19

DEBUG:jsonlitedb-sql:COMMIT


Of course, you can bulk delete by the query which will delete all matching items and not even be in the loop

In [12]:
db.delete(db.Q.last == "Harrison")

DEBUG:jsonlitedb-sql:BEGIN 
DEBUG:jsonlitedb-sql:
DELETE FROM items 
WHERE
    ( JSON_EXTRACT(data, '$."last"') = 'Harrison' )

DEBUG:jsonlitedb-sql:COMMIT


### Not optimized

In [13]:
row_ids = [row.rowid for row in db.query(db.Q.last == "McCartney")]
for row_id in row_ids:
    db.delete_by_rowid(row_id)

DEBUG:jsonlitedb-sql:
SELECT rowid, data FROM items 
WHERE
    ( JSON_EXTRACT(data, '$."last"') = 'McCartney' )


DEBUG:jsonlitedb-sql:BEGIN 
DEBUG:jsonlitedb-sql:
DELETE FROM items 
WHERE
    rowid = 2

DEBUG:jsonlitedb-sql:COMMIT
DEBUG:jsonlitedb-sql:BEGIN 
DEBUG:jsonlitedb-sql:
DELETE FROM items 
WHERE
    rowid = 7

DEBUG:jsonlitedb-sql:COMMIT
DEBUG:jsonlitedb-sql:BEGIN 
DEBUG:jsonlitedb-sql:
DELETE FROM items 
WHERE
    rowid = 12

DEBUG:jsonlitedb-sql:COMMIT
DEBUG:jsonlitedb-sql:BEGIN 
DEBUG:jsonlitedb-sql:
DELETE FROM items 
WHERE
    rowid = 17

DEBUG:jsonlitedb-sql:COMMIT


In [14]:
list(db)

DEBUG:jsonlitedb-sql:SELECT rowid, data FROM items
DEBUG:jsonlitedb-sql:SELECT COUNT(rowid) FROM items


[{'first': 'John', 'last': 'Lennon', 'born': 1940, 'role': 'guitar'},
 {'first': 'John', 'last': 'Lennon', 'born': 1940, 'role': 'guitar'},
 {'first': 'John', 'last': 'Lennon', 'born': 1940, 'role': 'guitar'},
 {'first': 'John', 'last': 'Lennon', 'born': 1940, 'role': 'guitar'}]